In [2]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.3MB/s ta 0:00:011
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

url = 'https://untappd.com/search?q=Murican+Pilsner'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get(url, headers=headers)
soup = response.content

In [ ]:
soupy = BeautifulSoup(soup, 'lxml')
print(soupy.prettify())

In [ ]:
beer_items = soupy.findAll(class_='beer-item')
len(beer_items)
for beer in beer_items:
    print(beer.findAll(class_="name")[0].find('a').text.strip())
    print(beer.findAll(class_='abv')[0].text.strip())
    print(beer.findAll(class_='rating')[0].text.strip())
    print(beer.findAll(class_='style')[0].text.strip())

    print()

In [ ]:
beer_items = soupy.findAll(class_='beer-item')
len(beer_items)
for beer in beer_items:
    print(beer.findAll(class_="name")[0].find('a').text.strip())
    print(beer.findAll(class_='abv')[0].text.strip())
    print(beer.findAll(class_='rating')[0].text.strip())
    print(beer.findAll(class_='style')[0].text.strip())

    print()

In [4]:
df_beers = pd.read_csv('data/beers.csv')
df_beers.head()

,Unnamed: 0,beer_id,name,display_name,abv,style_id,year,glass_id,organic,brewery_id,retired,status
0,0,c4f2KE,'Murican Pilsner,'Murican Pilsner,5.5,98.0,NaN,4.0,N,nHLlnK,N,verified
1,1,zTTWa2,11.5° PLATO,11.5° PLATO,4.5,164.0,NaN,NaN,N,nHLlnK,N,verified
2,2,zfP2fK,12th Of Never,12th Of Never,5.5,25.0,NaN,NaN,N,nLsoQ9,N,verified
3,3,xwYSL2,15th Anniversary Ale,15th Anniversary Ale,NaN,5.0,NaN,NaN,N,TMc6H2,N,verified
4,4,UJGpVS,16 So Fine Red Wheat Wine,16 So Fine Red Wheat Wine,11.0,35.0,NaN,NaN,N,TMc6H2,N,verified


In [ ]:
ids = []
ratings = []
errors = []
for i in tqdm(range(df_beers.shape[0])):
#     print(df_beers.loc[i,'display_name'])
    url = 'https://untappd.com/search?q='
    url = url + str(df_beers.loc[i,'display_name'])
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = response.content
    soupy = BeautifulSoup(soup, 'lxml')
    
    beer_items = soupy.findAll(class_='beer-item')
    for beer in beer_items:
        try:
            abv_raw = beer.findAll(class_='abv')[0].text.strip()
            abv_raw = abv_raw.strip('% ABV')
            abv = float(abv_raw)
        except Exception as e:
            print(e)
        if df_beers.loc[i,'abv'].astype(float) == abv:
            ids.append(df_beers.loc[i,'beer_id'])
            ratings.append(beer.findAll(class_='rating')[0].text.strip())
        elif df_beers.loc[i, 'name'].lower() == beer.findAll(class_="name")[0].find('a').text.strip().lower():
            ids.append(df_beers.loc[i,'beer_id'])
            ratings.append(beer.findAll(class_='rating')[0].text.strip())
        else:
            errors.append(df_beers.loc[i,'beer_id'])
        

In [ ]:
len(errors)

In [ ]:
len(ids)

In [ ]:
df_beers[df_beers['beer_id'].apply(lambda x: x in errors)]

In [ ]:
# ratings=ratings.strip('(')
# ratings=ratings.strip(')')

beer_ratings = pd.DataFrame()
beer_ratings['beer_id']=ids
beer_ratings['ratings']=ratings
beer_ratings['ratings']=beer_ratings['ratings'].apply(lambda x: x.strip('('))
beer_ratings['ratings']=beer_ratings['ratings'].apply(lambda x: x.strip(')'))
beer_ratings

In [33]:
ids2 = []
ratings2 = []
errors2 = []
df_untappd = pd.DataFrame()
untappd_columns = ['id','name', 'brewery', 'style', 'abv', 'ibu', 'rating']
for col in untappd_columns:
    df_untappd[col]=''

count=0    
for i in tqdm(range(df_beers.shape[0])):
#     print(df_beers.loc[i,'display_name'])
    url = 'https://untappd.com/search?q='
    url = url + str(df_beers.loc[i,'display_name'])
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = response.content
    soupy = BeautifulSoup(soup, 'lxml')
    
    beer_items = soupy.findAll(class_='beer-item')
#     print(soupy)
    for beer in beer_items:
#         print(beer)
#         try:
#             abv_raw = beer.findAll(class_='abv')[0].text.strip()
#             abv_raw = abv_raw.strip('% ABV')
#             abv = float(abv_raw)
#         except:
#             abv=0
#         for feature in ['name', 'brewery', 'style', 'abv', 'ibu', 'rating']:
#             df_untappd.set_value(count, 'id', df_beers.loc[i,'beer_id']) 
#             print(feature)
#             print(beer.findAll(class_=feature)[0].find('a').text.strip())
#             try:
#                 df_untappd.set_value(count, feature, beer.findAll(class_=feature)[0].find('a').text.strip())
#             except:
#                 df_untappd.set_value(count, feature, None)

        df_untappd.set_value(count, 'id', df_beers.loc[i,'beer_id']) 
#         print(beer.findAll(class_="name")[0].find('a').text.strip())
        df_untappd.set_value(count, 'name', beer.findAll(class_="name")[0].find('a').text.strip())
#         print(beer.findAll(class_="brewery")[0].find('a').text.strip())
        df_untappd.set_value(count, 'brewery', beer.findAll(class_="brewery")[0].find('a').text.strip())
#         print(beer.findAll(class_="style")[0].text)
        df_untappd.set_value(count, 'style', beer.findAll(class_="style")[0].text.strip())
#         print(beer.findAll(class_="abv")[0].text.strip())
        df_untappd.set_value(count, 'abv', beer.findAll(class_="abv")[0].text.strip())
#         print(beer.findAll(class_="ibu")[0].text.strip())
        df_untappd.set_value(count, 'ibu', beer.findAll(class_="ibu")[0].text.strip())
#         print(beer.findAll(class_="rating")[0].text.strip())
        df_untappd.set_value(count, 'rating', beer.findAll(class_="rating")[0].text.strip())
        
        count +=1

  0%|          | 0/1109 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:38: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:40: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:46: 

In [34]:
df_untappd.shape

(324, 7)

In [35]:
df_untappd.tail()

,id,name,brewery,style,abv,ibu,rating
319,ZzpzSy,Tickled Pink,Oskar Blues Brewery,Saison / Farmhouse Ale,6.6% ABV,25 IBU,(3.527)
320,ZzpzSy,Tickled Pink,Fort Collins Brewery (FCB),Sour - Gose,5.5% ABV,45 IBU,(3.606)
321,ZzpzSy,Tickled Pink,Rhombus Guys Brewing Company,Sour - Fruited,5% ABV,3 IBU,(3.826)
322,ZzpzSy,Tickled Pink,Call To Arms Brewing Company,Saison / Farmhouse Ale,7.6% ABV,N/A IBU,(3.875)
323,ZzpzSy,Tickled Pink,Garage Project,Sour - Ale,5.7% ABV,N/A IBU,(0)


In [42]:
df_untappd.to_csv('untappd_beer_ratings.csv')

In [41]:
df_beers[df_beers['beer_id'].apply(lambda x: x not in df_untappd['id'].unique())]

,Unnamed: 0,beer_id,name,display_name,abv,style_id,year,glass_id,organic,brewery_id,retired,status
5,5,vz5JZ1,1794 The Fergal Project,1794 The Fergal Project,4.50,42.0,NaN,NaN,N,DifSi4,N,verified
28,28,lWygSS,471 Double IPA - Hull Melon,471 Double IPA - Hull Melon,9.20,31.0,NaN,NaN,N,IImUD9,N,verified
29,29,fa0oqf,471 ESB - Extra Special Bitter,471 ESB - Extra Special Bitter,7.80,5.0,NaN,5.0,N,IImUD9,Y,verified
32,32,tw2Iw0,471 IPA. Aggressive Hoppiness,471 IPA. Aggressive Hoppiness,9.20,31.0,NaN,NaN,N,IImUD9,Y,verified
33,33,GYF0P4,471 Pilsner,471 Pilsner,NaN,98.0,NaN,NaN,N,IImUD9,N,verified
37,37,Fhw2NF,7 Cities Pilsner,7 Cities Pilsner,5.00,98.0,NaN,NaN,Y,p1tFbP,N,verified
49,49,aG4Ie2,Alpha Dog Imperial IPA,Alpha Dog Imperial IPA,8.50,31.0,NaN,NaN,N,yX6twV,N,verified
50,50,hYaduh,Alt Route - Beer Camp Across America,Alt Route - Beer Camp Across America (2014),6.60,55.0,2014.0,5.0,N,nHLlnK,N,verified
52,52,qIa0fL,Amber Beer,Amber Beer,NaN,32.0,NaN,NaN,N,p3YrOa,N,verified
53,53,Zd8Cxd,American Summer Hoppy Wit,American Summer Hoppy Wit,6.00,65.0,NaN,NaN,N,q6vJUK,N,verified
